In [1]:
import pandas as pd
import unicodedata
import requests

# #send to rest API
import json 
import requests
from requests.auth import HTTPBasicAuth

In [2]:
#batter_bases

In [3]:
under_runs_rate=pd.read_csv('under_runs_rate.csv')
over_runs_rate=pd.read_csv('over_runs_rate.csv')
def_rank = pd.read_csv('../scrapers/def_scraper/batter_runs_def_rank.csv')
batter_team_2025 = pd.read_csv('../batter_record_1_hit/stats/batters_team_2025.csv')

In [4]:
team_mapping_5 = {
    'CHC': 'CHC',
    'LAD': 'LAD',
    'ARI': 'ARI',
    'PIT': 'PIT',
    'MIA': 'MIA',
    'TOR': 'TOR',
    'BOS': 'BOS',
    'ATH': 'ATH',
    'BAL': 'BAL',
    'SEA': 'SEA',
    'SDP': 'SDP',
    'ATL': 'ATL',
    'PHI': 'PHI',
    'TEX': 'TEX',
    'MIL': 'MIL',
    'WSN': 'WSH',
    'NYY': 'NYY',
    'STL': 'STL',
    'TBR': 'TBR',
    'DET': 'DET',
    'KCR': 'KCR',
    'CLE': 'CLE',
    'COL': 'COL',
    'NYM': 'NYM',
    'CIN': 'CIN',
    'MIN': 'MIN',
    'SFG': 'SFG',
    'HOU': 'HOU',
    'CHW': 'CHW',
    'LAAs': 'LAA'
}

batter_team_2025['team'] = batter_team_2025['team'].map(team_mapping_5)

In [5]:
def_rank.rename(columns={'Team': 'team'}, inplace=True)

In [6]:
def_rank.rename(columns={'Rank': 'def_rank'}, inplace=True)

In [7]:
team_mapping = {
    'Milwaukee Brewers': 'MIL',
    'Minnesota Twins': 'MIN',
    'Chicago Cubs': 'CHC',
    'Colorado Rockies': 'COL',
    'Washington Nationals': 'WSH',
    'Toronto Blue Jays': 'TOR',
    'Atlanta Braves': 'ATL',
    'Baltimore Orioles': 'BAL',
    'New York Yankees': 'NYY',
    'Philadelphia Phillies': 'PHI',
    'Pittsburgh Pirates': 'PIT',
    'Kansas City Royals': 'KCR',
    'Arizona Diamondbacks': 'ARI',
    'Boston Red Sox': 'BOS',
    'Detroit Tigers': 'DET',
    'Cleveland Guardians': 'CLE',
    'Tampa Bay Rays': 'TBR',
    'San Francisco Giants': 'SFG',
    'Los Angeles Dodgers': 'LAD',
    'Seattle Mariners': 'SEA',
    'Cincinnati Reds': 'CIN',
    'Texas Rangers': 'TEX',
    'Chicago White Sox': 'CHW',
    'Los Angeles Angels': 'LAA',
    'Miami Marlins': 'MIA',
    'New York Mets': 'NYM',
    'Athletics': 'OAK',
    'St. Louis Cardinals': 'STL',
    'San Diego Padres': 'SDP',
    'Houston Astros': 'HOU'
}

def_rank['team'] = def_rank['team'].map(team_mapping)

In [8]:
#fixing the accentes on the names
def normalize_name(name):
    if not isinstance(name, str):
        return name
    return unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8')

In [9]:
batter_team_2025['player'] = batter_team_2025['player'].apply(normalize_name)

In [10]:
#cleaning the # and *
def clean_name_symbols(name):
    if not isinstance(name, str):
        return name
    return name.replace('*', '').replace('#', '')

batter_team_2025['player'] = batter_team_2025['player'].apply(clean_name_symbols)

In [11]:
# Rows to delete
rows_to_drop = set()

# Loop through the DataFrame
for i, val in batter_team_2025['team'].items():
    if val == '2TM':
        rows_to_drop.update(range(i, i+2))  # this row + 1 more
    elif val == '3TM':
        rows_to_drop.update(range(i, i+3))  # this row + 2 more
    elif val == '4TM':
        rows_to_drop.update(range(i, i+4))  # this row + 3 more
    elif val == '5TM':
        rows_to_drop.update(range(i, i+5))  # this row + 3 more

# Drop rows
batter_team_2025 = batter_team_2025.drop(rows_to_drop).reset_index(drop=True)

In [12]:
headers = {
    'X-Api-Key': 'fac7e1a69f2baa47da395bde36dc869c'  
    }

url = 'https://api.sportsgameodds.com/v2/events/'

next_cursor = None
event_data = []

while True:
    try:
        response = requests.get(
            url,
            headers=headers,
            params={
                "sportID":"BASEBALL",
                "leagueID": "MLB",
                "oddIDs": "points-PLAYER_ID-game-ou-over",
                "includeOpposingOddIDs": "true", 
                "oddsAvailable":"true",
                "cursor": next_cursor,
                "include":"teams",
                'started':"false",

            },
        )

        response.raise_for_status()  

        data = response.json()
        event_data.extend(data.get("data", []))
        next_cursor = data.get("nextCursor")

        if not next_cursor:
            break

    except requests.exceptions.RequestException as e:
        print("Error fetching events:", e)
        break



In [13]:
event_data

[{'eventID': 'n0C2NflrySgFACc5WKoP',
  'sportID': 'BASEBALL',
  'leagueID': 'MLB',
  'type': 'match',
  'odds': {'points-JACKSON_CHOURIO_1_MLB-game-ou-over': {'oddID': 'points-JACKSON_CHOURIO_1_MLB-game-ou-over',
    'opposingOddID': 'points-JACKSON_CHOURIO_1_MLB-game-ou-under',
    'marketName': 'Jackson Chourio Runs Over/Under',
    'statID': 'points',
    'statEntityID': 'JACKSON_CHOURIO_1_MLB',
    'periodID': 'game',
    'betTypeID': 'ou',
    'sideID': 'over',
    'playerID': 'JACKSON_CHOURIO_1_MLB',
    'started': False,
    'ended': False,
    'cancelled': False,
    'bookOddsAvailable': True,
    'fairOddsAvailable': True,
    'fairOdds': '-107',
    'bookOdds': '-125',
    'fairOverUnder': '0.5',
    'bookOverUnder': '0.5',
    'openFairOdds': '-107',
    'openBookOdds': '-125',
    'openFairOverUnder': '0.5',
    'openBookOverUnder': '0.5',
    'scoringSupported': True,
    'byBookmaker': {'fanduel': {'odds': '-130',
      'overUnder': '0.5',
      'lastUpdatedAt': '2025-05-

In [14]:
odds_list = []

for event in event_data:
    event_id = event.get("eventID", "Unknown Event")
    teams = sorted(set(player_data.get("teamID") for player_data in event.get("players", {}).values() if player_data.get("teamID")))
    
    odds = event.get("odds", {})
    
    for odd_id, odd_object in odds.items():
        base_row = {
            "eventID": event_id,
            "teams": ", ".join(teams),
            "oddID": odd_object.get("oddID"),
            "opposingOddID": odd_object.get("opposingOddID"),
            "marketName": odd_object.get("marketName"),
            "playerID": odd_object.get("statEntityID"),
            "side": odd_object.get("sideID"),
            "line": odd_object.get("bookOverUnder", odd_object.get("fairOverUnder", "N/A")),
            "score": odd_object.get("score", "N/A"),
        }
        
        by_bookmaker = odd_object.get("byBookmaker", {})
        
        for bookmaker, book_data in by_bookmaker.items():
            row = base_row.copy()
            row["bookmaker"] = bookmaker
            row["odds"] = book_data.get("odds", "N/A")
            row["overUnder"] = book_data.get("overUnder", "N/A")
            odds_list.append(row)

df_odds = pd.DataFrame(odds_list)


In [15]:
df_odds

,eventID,teams,oddID,opposingOddID,marketName,playerID,side,line,score,bookmaker,odds,overUnder
0,n0C2NflrySgFACc5WKoP,"CLEVELAND_GUARDIANS_MLB, MILWAUKEE_BREWERS_MLB",points-JACKSON_CHOURIO_1_MLB-game-ou-over,points-JACKSON_CHOURIO_1_MLB-game-ou-under,Jackson Chourio Runs Over/Under,JACKSON_CHOURIO_1_MLB,over,0.5,N/A,fanduel,-130,0.5
1,n0C2NflrySgFACc5WKoP,"CLEVELAND_GUARDIANS_MLB, MILWAUKEE_BREWERS_MLB",points-JACKSON_CHOURIO_1_MLB-game-ou-over,points-JACKSON_CHOURIO_1_MLB-game-ou-under,Jackson Chourio Runs Over/Under,JACKSON_CHOURIO_1_MLB,over,0.5,N/A,hardrockbet,-130,0.5
2,n0C2NflrySgFACc5WKoP,"CLEVELAND_GUARDIANS_MLB, MILWAUKEE_BREWERS_MLB",points-JACKSON_CHOURIO_1_MLB-game-ou-over,points-JACKSON_CHOURIO_1_MLB-game-ou-under,Jackson Chourio Runs Over/Under,JACKSON_CHOURIO_1_MLB,over,0.5,N/A,draftkings,-130,0.5
3,n0C2NflrySgFACc5WKoP,"CLEVELAND_GUARDIANS_MLB, MILWAUKEE_BREWERS_MLB",points-JACKSON_CHOURIO_1_MLB-game-ou-over,points-JACKSON_CHOURIO_1_MLB-game-ou-under,Jackson Chourio Runs Over/Under,JACKSON_CHOURIO_1_MLB,over,0.5,N/A,underdog,+100,0.5
4,n0C2NflrySgFACc5WKoP,"CLEVELAND_GUARDIANS_MLB, MILWAUKEE_BREWERS_MLB",points-JACKSON_CHOURIO_1_MLB-game-ou-over,points-JACKSON_CHOURIO_1_MLB-game-ou-under,Jackson Chourio Runs Over/Under,JACKSON_CHOURIO_1_MLB,over,0.5,N/A,prizepicks,+100,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...
3691,yfO7V46ops0LswTPjvfe,"ARIZONA_DIAMONDBACKS_MLB, SAN_FRANCISCO_GIANTS...",points-PATRICK_BAILEY_1_MLB-game-ou-under,points-PATRICK_BAILEY_1_MLB-game-ou-over,Patrick Bailey Runs Over/Under,PATRICK_BAILEY_1_MLB,under,0.5,N/A,bet365,-240,0.5
3692,yfO7V46ops0LswTPjvfe,"ARIZONA_DIAMONDBACKS_MLB, SAN_FRANCISCO_GIANTS...",points-PATRICK_BAILEY_1_MLB-game-ou-under,points-PATRICK_BAILEY_1_MLB-game-ou-over,Patrick Bailey Runs Over/Under,PATRICK_BAILEY_1_MLB,under,0.5,N/A,hardrockbet,-250,0.5
3693,yfO7V46ops0LswTPjvfe,"ARIZONA_DIAMONDBACKS_MLB, SAN_FRANCISCO_GIANTS...",points-PATRICK_BAILEY_1_MLB-game-ou-under,points-PATRICK_BAILEY_1_MLB-game-ou-over,Patrick Bailey Runs Over/Under,PATRICK_BAILEY_1_MLB,under,0.5,N/A,caesars,-254,0.5
3694,yfO7V46ops0LswTPjvfe,"ARIZONA_DIAMONDBACKS_MLB, SAN_FRANCISCO_GIANTS...",points-PATRICK_BAILEY_1_MLB-game-ou-under,points-PATRICK_BAILEY_1_MLB-game-ou-over,Patrick Bailey Runs Over/Under,PATRICK_BAILEY_1_MLB,under,0.5,N/A,betmgm,-250,0.5


In [16]:
#getting the last 3 letters of the teams collumn to get MLB
df_odds['sport'] = df_odds['teams'].str[-3:]

In [17]:
# Split into two new columns
df_odds[['team_1', 'team_2']] = df_odds['teams'].str.split(', ', expand=True)
# Remove the "_MLB" suffix from both columns using .loc
df_odds.loc[:, 'team_1'] = df_odds['team_1'].str.replace('_MLB', '', regex=False)
df_odds.loc[:, 'team_2'] = df_odds['team_2'].str.replace('_MLB', '', regex=False)
# Drop the original 'teams' column
df_odds = df_odds.drop(columns=['teams'])

In [18]:
#extract the players name from marketName
df_odds['player'] = df_odds['marketName'].str.extract(r'^(.*?)\s*Runs')

In [19]:
# Define the mapping dictionary
team_mapping = {
    'ARIZONA_DIAMONDBACKS': 'ARI',
    'ATLANTA_BRAVES': 'ATL',
    'BALTIMORE_ORIOLES': 'BAL',
    'BOSTON_RED_SOX': 'BOS',
    'CHICAGO_CUBS': 'CHC',
    'CHICAGO_WHITE_SOX': 'CHW',
    'CINCINNATI_REDS': 'CIN',
    'CLEVELAND_GUARDIANS': 'CLE',
    'COLORADO_ROCKIES': 'COL',
    'DETROIT_TIGERS': 'DET',
    'HOUSTON_ASTROS': 'HOU',
    'KANSAS_CITY_ROYALS': 'KCR',
    'LOS_ANGELES_ANGELS': 'LAA',
    'LOS_ANGELES_DODGERS': 'LAD',
    'MIAMI_MARLINS': 'MIA',
    'MILWAUKEE_BREWERS': 'MIL',
    'MINNESOTA_TWINS': 'MIN',
    'NEW_YORK_METS': 'NYM',
    'NEW_YORK_YANKEES': 'NYY',
    'OAKLAND_ATHLETICS': 'OAK',
    'PHILADELPHIA_PHILLIES': 'PHI',
    'PITTSBURGH_PIRATES': 'PIT',
    'SAN_DIEGO_PADRES': 'SDP',
    'SAN_FRANCISCO_GIANTS': 'SFG',
    'SEATTLE_MARINERS': 'SEA',
    'STLOUIS_CARDINALS': 'STL',
    'TAMPA_BAY_RAYS': 'TBR',
    'TEXAS_RANGERS': 'TEX',
    'TORONTO_BLUE_JAYS': 'TOR',
    'WASHINGTON_NATIONALS': 'WSH'}


# Apply the mapping
df_odds.loc[:, 'team_1'] = df_odds['team_1'].map(team_mapping)
df_odds.loc[:, 'team_2'] = df_odds['team_2'].map(team_mapping)

In [20]:
#making the line collumn an integer
df_odds['overUnder'] = pd.to_numeric(df_odds['overUnder'], errors='coerce')

In [21]:
df_odds['bet']='runs'

In [22]:
df_odds=df_odds[['player','side','overUnder','bookmaker','odds','sport','team_1','team_2','bet']]

In [23]:
# Merge on the 'player' column
df_odds = df_odds.merge(
    batter_team_2025[['player', 'team']],
    on='player',
    how='left'
)


In [24]:
df_odds.dropna()

,player,side,overUnder,bookmaker,odds,sport,team_1,team_2,bet,team
0,Jackson Chourio,over,0.5,fanduel,-130,MLB,CLE,MIL,runs,MIL
1,Jackson Chourio,over,0.5,hardrockbet,-130,MLB,CLE,MIL,runs,MIL
2,Jackson Chourio,over,0.5,draftkings,-130,MLB,CLE,MIL,runs,MIL
3,Jackson Chourio,over,0.5,underdog,+100,MLB,CLE,MIL,runs,MIL
4,Jackson Chourio,over,0.5,prizepicks,+100,MLB,CLE,MIL,runs,MIL
...,...,...,...,...,...,...,...,...,...,...
3691,Patrick Bailey,under,0.5,bet365,-240,MLB,ARI,SFG,runs,SFG
3692,Patrick Bailey,under,0.5,hardrockbet,-250,MLB,ARI,SFG,runs,SFG
3693,Patrick Bailey,under,0.5,caesars,-254,MLB,ARI,SFG,runs,SFG
3694,Patrick Bailey,under,0.5,betmgm,-250,MLB,ARI,SFG,runs,SFG


In [25]:
# Step 1: Identify the "odd one out" team to find the defensive team
df_odds['def_team'] = df_odds.apply(lambda row: row['team_2'] if row['team'] == row['team_1'] else row['team_1'], axis=1)

In [26]:
df_odds.dropna()

,player,side,overUnder,bookmaker,odds,sport,team_1,team_2,bet,team,def_team
0,Jackson Chourio,over,0.5,fanduel,-130,MLB,CLE,MIL,runs,MIL,CLE
1,Jackson Chourio,over,0.5,hardrockbet,-130,MLB,CLE,MIL,runs,MIL,CLE
2,Jackson Chourio,over,0.5,draftkings,-130,MLB,CLE,MIL,runs,MIL,CLE
3,Jackson Chourio,over,0.5,underdog,+100,MLB,CLE,MIL,runs,MIL,CLE
4,Jackson Chourio,over,0.5,prizepicks,+100,MLB,CLE,MIL,runs,MIL,CLE
...,...,...,...,...,...,...,...,...,...,...,...
3691,Patrick Bailey,under,0.5,bet365,-240,MLB,ARI,SFG,runs,SFG,ARI
3692,Patrick Bailey,under,0.5,hardrockbet,-250,MLB,ARI,SFG,runs,SFG,ARI
3693,Patrick Bailey,under,0.5,caesars,-254,MLB,ARI,SFG,runs,SFG,ARI
3694,Patrick Bailey,under,0.5,betmgm,-250,MLB,ARI,SFG,runs,SFG,ARI


In [27]:
#drop redundant collumns that are useless
df_odds = df_odds.drop(columns=['team_1', 'team_2'], errors='ignore')

In [28]:
# Rename the team column before merging (if needed)
def_rank = def_rank.rename(columns={"team": "def_team"})

In [29]:
# Merge on the 'player' column
df_odds = df_odds.merge(def_rank, on='def_team', how='left')
df_odds.dropna()

,player,side,overUnder,bookmaker,odds,sport,bet,team,def_team,Unnamed: 0,def_rank
0,Jackson Chourio,over,0.5,fanduel,-130,MLB,runs,MIL,CLE,22,22
1,Jackson Chourio,over,0.5,hardrockbet,-130,MLB,runs,MIL,CLE,22,22
2,Jackson Chourio,over,0.5,draftkings,-130,MLB,runs,MIL,CLE,22,22
3,Jackson Chourio,over,0.5,underdog,+100,MLB,runs,MIL,CLE,22,22
4,Jackson Chourio,over,0.5,prizepicks,+100,MLB,runs,MIL,CLE,22,22
...,...,...,...,...,...,...,...,...,...,...,...
3691,Patrick Bailey,under,0.5,bet365,-240,MLB,runs,SFG,ARI,24,24
3692,Patrick Bailey,under,0.5,hardrockbet,-250,MLB,runs,SFG,ARI,24,24
3693,Patrick Bailey,under,0.5,caesars,-254,MLB,runs,SFG,ARI,24,24
3694,Patrick Bailey,under,0.5,betmgm,-250,MLB,runs,SFG,ARI,24,24


In [30]:
#making game collumn
df_odds['game'] = df_odds['team'] + ' vs ' + df_odds['def_team']

In [31]:
df_odds2=df_odds.dropna()

In [32]:
df_odds3 = df_odds2.drop(columns=['team', 'def_team'])

In [33]:
overs = df_odds3[(df_odds3['side'].str.lower() == 'over') & (df_odds3['def_rank'] > 15)]
unders = df_odds3[(df_odds3['side'].str.lower() == 'under') & (df_odds3['def_rank'] < 16)]

In [34]:
overs = overs.merge(over_runs_rate, on='player', how='left')
unders = unders.merge(under_runs_rate, on='player', how='left')

In [35]:
overs_2=overs.dropna()
unders_2=unders.dropna()

In [36]:
#making odds collumn numeric
unders_2['odds'] = pd.to_numeric(unders_2['odds'], errors='coerce')
overs_2['odds'] = pd.to_numeric(overs_2['odds'], errors='coerce')

C:\Users\Joshua\AppData\Local\Temp\ipykernel_123712\2110421890.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unders_2['odds'] = pd.to_numeric(unders_2['odds'], errors='coerce')
C:\Users\Joshua\AppData\Local\Temp\ipykernel_123712\2110421890.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overs_2['odds'] = pd.to_numeric(overs_2['odds'], errors='coerce')


In [37]:
#making decimal odds
unders_2['decimal_odds'] = unders_2['odds'].apply(
    lambda x: (x / 100) + 1 if x > 0 else (100 / abs(x)) + 1
)
overs_2['decimal_odds'] = overs_2['odds'].apply(
    lambda x: (x / 100) + 1 if x > 0 else (100 / abs(x)) + 1
)

C:\Users\Joshua\AppData\Local\Temp\ipykernel_123712\875580270.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unders_2['decimal_odds'] = unders_2['odds'].apply(
C:\Users\Joshua\AppData\Local\Temp\ipykernel_123712\875580270.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overs_2['decimal_odds'] = overs_2['odds'].apply(


In [38]:
#making profit
overs_2['profit'] = (overs_2['decimal_odds'] * 100) - 100
unders_2['profit'] = (unders_2['decimal_odds'] * 100) - 100

C:\Users\Joshua\AppData\Local\Temp\ipykernel_123712\3093691362.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overs_2['profit'] = (overs_2['decimal_odds'] * 100) - 100
C:\Users\Joshua\AppData\Local\Temp\ipykernel_123712\3093691362.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unders_2['profit'] = (unders_2['decimal_odds'] * 100) - 100


In [39]:
def calculate_ev(row):
    if row['overUnder'] == 0.5:
        rate = row['1_runs_rate_over']
    elif row['overUnder'] == 1.5:
        rate = row['2_runs_rate_over']
    elif row['overUnder'] == 2.5:
        rate = row['3_runs_rate_over']
    elif row['overUnder'] == 3.5:
        rate = row['4_runs_rate_over']
    else:
        return None  # or np.nan, if you want to leave it blank for other lines

    # Expected Value formula
    return (rate * row['profit']) - ((1 - rate) * 100)

# Apply the formula to each row
overs_2['edge'] = overs_2.apply(calculate_ev, axis=1)

C:\Users\Joshua\AppData\Local\Temp\ipykernel_123712\1516265758.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overs_2['edge'] = overs_2.apply(calculate_ev, axis=1)


In [40]:
#edge for unders
def calculate_under_ev(row):
    if row['overUnder'] == 0.5:
        rate = row['1_runs_rate_under']
    elif row['overUnder'] == 1.5:
        rate = row['2_runs_rate_under']
    elif row['overUnder'] == 2.5:
        rate = row['3_runs_rate_under']
    elif row['overUnder'] == 3.5:
        rate = row['4_runs_rate_under']
    else:
        return None  # or np.nan for unsupported lines

    return ((1 - rate) * row['profit']) - (rate * 100)

# Apply to each row
unders_2['edge'] = unders_2.apply(calculate_under_ev, axis=1)

C:\Users\Joshua\AppData\Local\Temp\ipykernel_123712\4051107437.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unders_2['edge'] = unders_2.apply(calculate_under_ev, axis=1)


In [41]:
overs_2 = overs_2[overs_2['edge'] > 20]

In [42]:
unders_2 = unders_2[unders_2['edge'] > 20]

In [43]:
#merging overs_2 on top of unders_2
final_df = pd.concat([overs_2, unders_2], ignore_index=True)

In [44]:
final_df=final_df[['player','side','overUnder','bookmaker','odds','sport','bet','edge','game']]

In [45]:
final_df['bet'] = final_df['side'].astype(str) + ' ' + final_df['overUnder'].astype(str) + ' ' + final_df['bet'].astype(str)


In [46]:
final_df = final_df.rename(columns={'bookmaker': 'book'})

In [47]:
final_df['edge'] = final_df['edge'] / 2

In [48]:
final_df['edge'] = final_df['edge'].round(1)

In [49]:
final_df=final_df[['book','sport','player','game','bet','odds','edge']]

In [50]:
final_df = final_df.sort_values(by='edge', ascending=False).reset_index(drop=True)


In [51]:
final_df

,book,sport,player,game,bet,odds,edge
0,bet365,MLB,Lars Nootbaar,STL vs PHI,under 0.5 runs,-135,15.8
1,prizepicks,MLB,Jackson Chourio,MIL vs CLE,over 0.5 runs,100,15.1
2,underdog,MLB,Jackson Chourio,MIL vs CLE,over 0.5 runs,100,15.1
3,bet365,MLB,Dansby Swanson,CHC vs MIA,over 0.5 runs,140,15.0
4,hardrockbet,MLB,Lars Nootbaar,STL vs PHI,under 0.5 runs,-140,14.8
...,...,...,...,...,...,...,...
71,hardrockbet,MLB,Vinnie Pasquantino,KCR vs HOU,under 0.5 runs,-160,10.1
72,draftkings,MLB,Vinnie Pasquantino,KCR vs HOU,under 0.5 runs,-160,10.1
73,betmgm,MLB,Vinnie Pasquantino,KCR vs HOU,under 0.5 runs,-160,10.1
74,espnbet,MLB,Connor Wong,BOS vs DET,under 0.5 runs,-180,10.0


In [52]:
#putting + in front of the ones that dont have it
final_df['odds'] = final_df['odds'].apply(lambda x: f"+{x}" if x > 0 else str(x))

final_df

,book,sport,player,game,bet,odds,edge
0,bet365,MLB,Lars Nootbaar,STL vs PHI,under 0.5 runs,-135,15.8
1,prizepicks,MLB,Jackson Chourio,MIL vs CLE,over 0.5 runs,+100,15.1
2,underdog,MLB,Jackson Chourio,MIL vs CLE,over 0.5 runs,+100,15.1
3,bet365,MLB,Dansby Swanson,CHC vs MIA,over 0.5 runs,+140,15.0
4,hardrockbet,MLB,Lars Nootbaar,STL vs PHI,under 0.5 runs,-140,14.8
...,...,...,...,...,...,...,...
71,hardrockbet,MLB,Vinnie Pasquantino,KCR vs HOU,under 0.5 runs,-160,10.1
72,draftkings,MLB,Vinnie Pasquantino,KCR vs HOU,under 0.5 runs,-160,10.1
73,betmgm,MLB,Vinnie Pasquantino,KCR vs HOU,under 0.5 runs,-160,10.1
74,espnbet,MLB,Connor Wong,BOS vs DET,under 0.5 runs,-180,10.0


In [53]:
# Convert to JSON list of dicts
data_json = json.loads(final_df.to_json(orient="records"))

# WordPress REST API endpoint
url = "https://pitstoppicks.com/wp-json/custom-api/v1/mlb_batter_runs"

# Headers
headers = {
    "Content-Type": "application/json"
}

# Send POST request (no auth needed)
response = requests.post(url, headers=headers, json=data_json)

# Print response for debugging
print("Status Code:", response.status_code)
print("Response Text:", response.text)

Status Code: 200
Response Text: {"message":"Data received","data":[{"book":"bet365","sport":"MLB","player":"Lars Nootbaar","game":"STL vs PHI","bet":"under 0.5 runs","odds":"-135","edge":15.8},{"book":"prizepicks","sport":"MLB","player":"Jackson Chourio","game":"MIL vs CLE","bet":"over 0.5 runs","odds":"+100","edge":15.1},{"book":"underdog","sport":"MLB","player":"Jackson Chourio","game":"MIL vs CLE","bet":"over 0.5 runs","odds":"+100","edge":15.1},{"book":"bet365","sport":"MLB","player":"Dansby Swanson","game":"CHC vs MIA","bet":"over 0.5 runs","odds":"+140","edge":15},{"book":"hardrockbet","sport":"MLB","player":"Lars Nootbaar","game":"STL vs PHI","bet":"under 0.5 runs","odds":"-140","edge":14.8},{"book":"espnbet","sport":"MLB","player":"Lars Nootbaar","game":"STL vs PHI","bet":"under 0.5 runs","odds":"-140","edge":14.8},{"book":"fourwinds","sport":"MLB","player":"Lars Nootbaar","game":"STL vs PHI","bet":"under 0.5 runs","odds":"-141","edge":14.6},{"book":"betparx","sport":"MLB","pla

In [54]:
# # Convert DataFrame to JSON (ensure it's a Python list, not a string)
# data_json = json.loads(final_df.to_json(orient="records"))  # ✅ Converts string to Python object

# # WordPress API URL
# url = "https://pitstoppicks.com/wp-json/custom-api/v1/mlb_batter_total_runs"

# # Authentication
# # auth = HTTPBasicAuth("josheskragmailcom", "VKdW RLJu TlWM nE7Z odqH SjYg")
# auth = HTTPBasicAuth("josheskragmailcom", "BFgx z2iv nW9I voXn 1X51 fjRq")
# # Headers
# headers = {
#     "Content-Type": "application/json"
# }

# # Send POST request (requests will handle JSON encoding automatically)
# response = requests.post(url, headers=headers, auth=auth, json=data_json)  # ✅ FIX: Use json= instead of data=

# # Debugging Output
# print("Status Code:", response.status_code)
# print("Response Text:", response.text)

In [55]:
# Step 1: Sort to ensure reproducibility in tie cases
df_sorted = final_df.sort_values('edge')

# Step 2: Define a function to get the row with median edge for each group
def get_median_row(group):
    median_edge = group['edge'].median()
    # Get the row closest to the median (or first in case of exact match/ties)
    return group.iloc[(group['edge'] - median_edge).abs().argsort().iloc[0]]

# Step 3: Apply it to each group
df_filtered = df_sorted.groupby(['player', 'game', 'bet'], group_keys=False).apply(get_median_row)

# Step 4: Reset index if needed
df_filtered.reset_index(drop=True, inplace=True)
df_filtered

C:\Users\Joshua\AppData\Local\Temp\ipykernel_123712\1266780210.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filtered = df_sorted.groupby(['player', 'game', 'bet'], group_keys=False).apply(get_median_row)


,book,sport,player,game,bet,odds,edge
0,espnbet,MLB,Alec Bohm,PHI vs STL,under 0.5 runs,-155,10.2
1,hardrockbet,MLB,Alex Bregman,BOS vs DET,under 0.5 runs,-145,12.0
2,bet365,MLB,Christian Walker,HOU vs KCR,under 0.5 runs,-135,10.3
3,bet365,MLB,Connor Wong,BOS vs DET,under 0.5 runs,-180,10.0
4,caesars,MLB,Dansby Swanson,CHC vs MIA,over 0.5 runs,+126,11.2
5,hardrockbet,MLB,Dillon Dingler,DET vs BOS,under 0.5 runs,-240,12.0
6,underdog,MLB,Jackson Chourio,MIL vs CLE,over 0.5 runs,+100,15.1
7,bet365,MLB,Jackson Merrill,SDP vs LAA,over 0.5 runs,+150,11.3
8,betmgm,MLB,Jake Cronenworth,SDP vs LAA,over 0.5 runs,+190,10.7
9,underdog,MLB,Jorge Polanco,SEA vs NYY,under 0.5 runs,+100,14.1


In [56]:
# Make sure odds column is numeric
df_filtered['odds'] = pd.to_numeric(df_filtered['odds'], errors='coerce')
# Convert American odds to decimal odds
def american_to_decimal(odds):
    if pd.isna(odds):
        return None
    if odds > 0:
        return round((odds / 100) + 1, 3)
    else:
        return round((100 / abs(odds)) + 1, 3)

# Apply conversion
df_filtered['decimal_odds'] = df_filtered['odds'].apply(american_to_decimal)

# Check it out
print(df_filtered[['odds', 'decimal_odds']].head())

   odds  decimal_odds
0  -155         1.645
1  -145         1.690
2  -135         1.741
3  -180         1.556
4   126         2.260


In [57]:
df_filtered['profit'] = (df_filtered['decimal_odds'] * 100) - 100
df_filtered

,book,sport,player,game,bet,odds,edge,decimal_odds,profit
0,espnbet,MLB,Alec Bohm,PHI vs STL,under 0.5 runs,-155,10.2,1.645,64.5
1,hardrockbet,MLB,Alex Bregman,BOS vs DET,under 0.5 runs,-145,12.0,1.690,69.0
2,bet365,MLB,Christian Walker,HOU vs KCR,under 0.5 runs,-135,10.3,1.741,74.1
3,bet365,MLB,Connor Wong,BOS vs DET,under 0.5 runs,-180,10.0,1.556,55.6
4,caesars,MLB,Dansby Swanson,CHC vs MIA,over 0.5 runs,126,11.2,2.260,126.0
5,hardrockbet,MLB,Dillon Dingler,DET vs BOS,under 0.5 runs,-240,12.0,1.417,41.7
6,underdog,MLB,Jackson Chourio,MIL vs CLE,over 0.5 runs,100,15.1,2.000,100.0
7,bet365,MLB,Jackson Merrill,SDP vs LAA,over 0.5 runs,150,11.3,2.500,150.0
8,betmgm,MLB,Jake Cronenworth,SDP vs LAA,over 0.5 runs,190,10.7,2.900,190.0
9,underdog,MLB,Jorge Polanco,SEA vs NYY,under 0.5 runs,100,14.1,2.000,100.0
